<h3>0.0 CARGA DE BIBLIOTECAS, VARIABLES, CLASES Y MÉTODOS IMPORTANTES</h3>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout, LSTM, Dense, Embedding
import pickle
import json
import os

num_lilith = '04'
archivo_json = '../../Chessia_Data_Convert/Target Data/Datos-V2.json'
tokens_lilith = "tokens"+num_lilith+".pkl"

class Partida:
    def __init__(self):
        self.movimientos = []
        self.resultado = None

# Función para cargar partidas en lotes
def cargar_partidas_por_rango(archivo_json, inicio=0, fin=103350):
    total_movimientos = []
    total_resultados = []
    with open(archivo_json, 'r') as file:
        # Carga el contenido completo como una lista de objetos JSON
        lista_json = json.load(file)
        
        # Itera sobre cada objeto JSON en la lista
        for i in range(inicio, fin):
            total_movimientos.append(lista_json[i]['partida']['movimientos'])
            total_resultados.append(lista_json[i]['partida']['resultado'])
            
    return total_movimientos, total_resultados

class PlotMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        try:
            if self.model.history:
                plt.figure(figsize=(10, 5))
                plt.subplot(1, 2, 1)
                plt.plot(self.model.history.history['loss'], label='Training Loss')
                plt.plot(self.model.history.history['val_loss'], label='Validation Loss')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.title('Loss Over Time')
                plt.legend()

                plt.subplot(1, 2, 2)
                plt.plot(self.model.history.history['accuracy'], label='Training Accuracy')
                plt.plot(self.model.history.history['val_accuracy'], label='Validation Accuracy')
                plt.xlabel('Epoch')
                plt.ylabel('Accuracy')
                plt.title('Accuracy Over Time')
                plt.legend()

        except Exception as e:
            print("Error al mostrar la gráfica:", e)

<h3>3.0 PROCESADO DE DATOS DE LILITH - USA ProcesadorPGN_V2</h3>

In [2]:
# Convertir el generador en una lista y extraer la primera tupla
recogido = cargar_partidas_por_rango(archivo_json, fin=100000)
movimientos = recogido[0]
resultados = recogido[1]
# Crear y ajustar el tokenizador
tokenizer = Tokenizer()
tokenizer.fit_on_texts(movimientos)

#Borramos los antiguos tokens si los hubiese
if os.path.exists(tokens_lilith):
    os.remove(tokens_lilith)

# Guardar el objeto Tokenizer en un archivo usando pickle
with open(tokens_lilith, "wb") as f:
    pickle.dump(tokenizer, f)

# Convertir todas las jugadas a secuencias de números enteros y luego a tensores válidos
movimientos_tokenized = tokenizer.texts_to_sequences(movimientos)
movimientos_tokenized = pad_sequences(movimientos_tokenized, maxlen=500, padding="pre")

resultados_tokenized = tf.one_hot([int(resultado) for resultado in resultados], depth=3)  # 3 clases: empate, victoria blanca, victoria negra

tensor_mov_entrena = tf.constant(movimientos_tokenized[:int(len(movimientos_tokenized) * 0.7) ])
tensor_res_entrena = tf.constant(resultados_tokenized [:int(len(resultados_tokenized)  * 0.7) ])
tensor_mov_prueba =  tf.constant(movimientos_tokenized[ int(len(movimientos_tokenized) * 0.7):])
tensor_res_prueba =  tf.constant(resultados_tokenized [ int(len(resultados_tokenized)  * 0.7):])

<h3>3.1 LILITH - DEFINICIÓN, ENTRENAMIENTO Y GRÁFICAS DE DATOS CON POCOS VALORES</h3>

<h4>&ensp;&ensp;3.1.1 CREACIÓN DEL MODELO</h4>

In [4]:
modelo = tf.keras.Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=256),
    LSTM(units=512, dropout=0.3, recurrent_dropout=0.3, return_sequences=True),
    LSTM(units=256, dropout=0.3, recurrent_dropout=0.3),
    Dense(units=256, activation='relu'),
    Dense(units=3, activation='softmax')
])

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
modelo.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

<h4>&ensp;&ensp;3.1.2 ENTRENAMIENTO</h4>

In [6]:
# Definir callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
checkpoint = ModelCheckpoint(filepath='modelo_checkpoint.keras', save_best_only=True)
plot_metrics = PlotMetricsCallback()

try:
    modelo = tf.keras.models.load_model('modelo_checkpoint.keras')
    print("Checkpoint cargado. Continuando entrenamiento desde el último punto guardado...\n\n")
except:
    print("No se encontró un checkpoint existente. Iniciando un nuevo entrenamiento...\n\n")

# Entrenar el modelo con los callbacks
history = modelo.fit(
    tensor_mov_entrena,
    tensor_res_entrena,
    epochs=15,
    initial_epoch=0,  # Especificar la primera época a ejecutar
    batch_size=256,
    validation_data=(tensor_mov_prueba, tensor_res_prueba),
    callbacks=[early_stopping, checkpoint]
)

# Guardar el modelo final
modelo.save("lilith"+num_lilith+".keras")

# Evaluar el modelo en los datos de prueba
loss, accuracy = modelo.evaluate(tensor_mov_prueba, tensor_res_prueba)
print("\n\nLoss en los datos de prueba:", loss)
print("Precisión en los datos de prueba:", accuracy)

No se encontró un checkpoint existente. Iniciando un nuevo entrenamiento...


Epoch 1/15
 41/274 ━━━━━━━━━━━━━━━━━━━━ 36:12 9s/step - accuracy: 0.4121 - loss: 1.0677

<h4>&ensp;&ensp;3.1.3 ESTADÍSTICAS Y GRÁFICAS</h4>

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

epocas = range(1, len(history.history['loss']) + 1)

# Graficar la precisión
axs[0].plot(epocas, history.history['accuracy'], 'r', label='Precisión en entrenamiento')
axs[0].plot(epocas, history.history['val_accuracy'], 'b', label='Precisión en validación')
axs[0].set_title('Precisión en entrenamiento y validación')
axs[0].set_xlabel('Épocas')
axs[0].set_ylabel('Precisión')
axs[0].legend()

# Graficar la pérdida
axs[1].plot(epocas, history.history['loss'], 'r', label='Pérdida en entrenamiento')
axs[1].plot(epocas, history.history['val_loss'], 'b', label='Pérdida en validación')
axs[1].set_title('Pérdida en entrenamiento y validación')
axs[1].set_xlabel('Épocas')
axs[1].set_ylabel('Pérdida')
axs[1].legend()

plt.tight_layout()  # Ajustar el diseño para evitar superposiciones
plt.show()

<h3>3.2 PRUEBA DE PREDICCIÓN LILITH</h3>

<h4>&ensp;&ensp;3.2.1 RECOGIDA DE DATOS Y TOKENS</h4>

In [3]:
# Convertir el generador en una lista y extraer la primera tupla
movimientos, resultados = cargar_partidas_por_rango(archivo_json, inicio=100000)

# Creación del tokenizer
nuevo_tokenizer = Tokenizer()

# Cargar el objeto Tokenizer desde el archivo
with open(tokens_lilith, "rb") as f:
    nuevo_tokenizer = pickle.load(f)

tensor_mov_pred = tf.constant(pad_sequences(nuevo_tokenizer.texts_to_sequences(movimientos), maxlen=500, padding="pre"))

<h4>&ensp;&ensp;3.2.2 PREDICCIÓN Y MUESTRA DE DATOS</h4>

In [ ]:
np.set_printoptions(suppress=True, precision=4)

lilith = tf.keras.models.load_model("lilith"+num_lilith+".keras")

res_pred = lilith.predict(tensor_mov_pred)
aciertos = 0

for i in range(len(res_pred)):
    if int(resultados[i]) != int(np.argmax(res_pred[i])): char = '❌'
    else: 
        char = '✅'
        aciertos += 1

    print(f"Predicción: {str(res_pred[i]):>22} -- Dice: {np.argmax(res_pred[i])} -- Real: {resultados[i]}  {char}  --- Aciertos: {aciertos:>5}/{i+1:<5} - {int(aciertos/(i+1)*100):>3}%")
